# 제로베이스데이터사이언스 스쿨 원데이 프로젝트
> ## 세계 테러 데이터 분석 - SQL 사용하기

---

#### 총 8문제 100점 만점 

- 문제 1 10점  
- 문제 2,3,5,6,7,8번 각 10점 
- 문제 4번 30점
- 총점 100점 

#### 유의사항

- 모든 정답 코드는 jupyter notebook 파일에 작성해주세요. 
- 1번 문제는 정답 코드를 그대로 공개해드립니다. 따라서 진행해주세요. 
- 그 외 문제들은 참고하실 수 있도록 주석과 출력 코드를 보여드립니다. 
- 제출시, 반드시 결과가 실행되어 있어야 합니다. 미실행 또는 오류시 감점입니다. 
- 제출시, [DS]sql3_choyongha.ipynb 형식에 맞춰 제출해주세요.


---

여러분은 이전 과제에서 세계 테러 데이터 분석 프로젝트를 진행하였습니다. 

이번 시간에는 분석할 데이터를 Database 로 변환하고, 분석에 필요한 데이터를 저장된 Database 에서 쿼리하여 가져와 보도록 하겠습니다. 

#### 작업환경

- aws rds 사용시: 이전 과제에서 생성한 AWS RDS (MySQL) 의 Database 및 사용자를 사용하세요. 
  - Database Name: oneday 
  - User Name / Password: oneday / 1234
- local 사용시: 로컬 환경에서 생성한 Database 및 사용자를 사용하세요.
  - 위와 동일하게 생성해주세요

## 원본 Data Download
- https://www.kaggle.com/datasets/START-UMD/gtd

---

문제 1.  csv 파일에 저장된 세계 테러 데이터를 하나의 테이블에 저장하세요. 

- globalterrorismmdb_0718.csv - https://www.kaggle.com/datasets/START-UMD/gtd
- pandas의 to_sql 함수 사용 (힌트 : create_engine)
- Table name : origin_terror_data 

제출 1. Table 생성 및 조회 결과 : DESC origin_terror_data, SELECT  COUNT(*) FROM origin_terror_data

제출 2. 관련 코드 및 실행 결과 (ipynb)



---

***해결못한 오류***
PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)


In [2]:
##############################################################################################
#  문제 1. 원본 데이터 (csv)를 데이터베이스로 변환하고 확인하기 
#  globalterrorismdb_0718dist.csv -> origin_terror_data (table)
#  Pandas 의 to_sql 사용 
##############################################################################################

##############################################################################################
#  pip install pymysql
#  pip install SQLAlchemy Flask-SQLAlchemy
##############################################################################################

import time
import pandas as pd
import pymysql
import configparser
import cryptography
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import text



# 데이터 불러오기
df = pd.read_csv('../globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')

# DB 접속 엔진 객체 생성
user = 'root'
password = 'oklp1349!?'
host = 'localhost'
port = 3306
database = 'oneday'


engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4')

# DB 테이블 명(생성될 테이블 이름)
table_name = "origin_terror_data"

connection = engine.connect()
transaction1 = connection.begin()

Session = sessionmaker(bind=engine)
session = Session()

try:
   
    df.to_sql(index = False, 
          name = table_name,
          con = engine,
          if_exists = 'append',
          method = 'multi', 
          chunksize = 10000)         
    session.commit()
   
except:
            # if any kind of exception occurs, rollback transaction
    session.rollback()
    raise
finally:
    session.close()


records = connection.execute(text("SELECT COUNT(*) FROM origin_terror_data")).fetchall()
print(records)



C:\Users\user\AppData\Local\Temp\ipykernel_11624\3432462479.py:26: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')


PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

문제 2. origin_terror_data 에서 region, country 관련 데이터는 code 와 txt (name) 속성으로 정의되어 있습니다. 

문제 2-1. Region 및 Country 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 
- Region 과 Country 데이터 사이의 관계를 파악하여 Foreign Key 를 설정하세요.

참고> 
- Region.region_code     = origin_terror_data.region
- Region.region_name     = origin_terror_data.region_txt
- Country.country_code   = origin_terror_data.country
- Country.country_name   = origin_terror_data.country_txt


In [4]:
##############################################################################################
#  문제 2. region / country / city 데이터 추출하여 데이터베이스로 변환하기 
#  중복을 제거한 code - name 값 
#  region - country - city 관계 정의 
##############################################################################################
import mysql.connector

conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "oklp1349!?",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)

**MAX(LENGTH())**

In [ ]:
# region_txt 의 max length 체크 
cursor.execute('select max(length(region_txt)) from origin_terror_data')

[(27,)]


In [ ]:
# Region 테이블 만들기 
cursor.execute('create table Region (region_code int, region_name varchar(27), primary key (region_code))')
cursor.execute('desc Region')
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)


[('region_code', b'int', 'NO', 'PRI', None, ''), ('region_name', b'varchar(32)', 'NO', '', None, '')]


In [ ]:
# country_txt 의 max length 체크 
cursor.execute('select max(length(country_txt)) from origin_terror_data')

[(32,)]


In [ ]:
# Country 테이블 만들기 
cursor.execute('create table Country (country_code int, region_code int, country_name varchar(32), primary key (country_code), foreign key(region_code) references origin_terror_data(region_code))')
cursor.execute('desc Country')
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)

[('country_code', b'int', 'NO', 'PRI', None, ''), ('region_code', b'int', 'NO', 'MUL', None, ''), ('country_name', b'varchar(32)', 'NO', '', None, '')]


문제 2-2. origin_terror_data 테이블에서 Region 및 Country 데이터를 추출하여 문제 2-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요. 

제출 3. Table 생성 및 조회 결과 : DESC Region, DESC Country

제출 4. 관련 코드 및 실행 결과 (ipynb)

**INSERT INTO 테이블 이름 SELECT 문**
- SQL에서 INSERT INTO SELECT 문은 한 테이블의 데이터를 다른 테이블로 복사할 때 자주 사용됩니다. 이를 통해 데이터 백업, 데이터 마이그레이션, 데이터 통합 등의 작업을 수행할 수 있습니다.
예를 들어, 기존 테이블의 데이터를 새로운 테이블로 옮기거나, 여러 테이블의 데이터를 하나의 테이블로 통합할 때 유용하게 사용됩니다.

In [ ]:
# Region 데이터 추출하기 
cursor.execute('select distinct region,region_txt from origin_terror_data')
result = cursor.fetchall()
result

[(1, 'North America'), (2, 'Central America & Caribbean'), (3, 'South America'), (4, 'East Asia'), (5, 'Southeast Asia'), (6, 'South Asia'), (7, 'Central Asia'), (8, 'Western Europe'), (9, 'Eastern Europe'), (10, 'Middle East & North Africa'), (11, 'Sub-Saharan Africa'), (12, 'Australasia & Oceania')]


In [ ]:
# Region 테이블에 INSERT 
for result_iterator in result:
    cursor.execute('insert into Region values (%s,%s)',result_iterator)
    cursor.commit()
cursor.close()

[(12,)]


In [ ]:
# Country 데이터 추출하기 
cursor.execute('select distinct country, region, country_txt from origin_terror_data')
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)

[(4, 6, 'Afghanistan'), (5, 9, 'Albania'), (6, 10, 'Algeria'), (7, 8, 'Andorra'), (8, 11, 'Angola'), (10, 2, 'Antigua and Barbuda'), (11, 3, 'Argentina'), (12, 7, 'Armenia'), (14, 12, 'Australia'), (15, 8, 'Austria'), (16, 7, 'Azerbaijan'), (17, 2, 'Bahamas'), (18, 10, 'Bahrain'), (19, 6, 'Bangladesh'), (20, 2, 'Barbados'), (21, 8, 'Belgium'), (22, 2, 'Belize'), (23, 11, 'Benin'), (25, 6, 'Bhutan'), (26, 3, 'Bolivia'), (28, 9, 'Bosnia-Herzegovina'), (29, 11, 'Botswana'), (30, 3, 'Brazil'), (31, 5, 'Brunei'), (32, 9, 'Bulgaria'), (33, 11, 'Burkina Faso'), (34, 11, 'Burundi'), (35, 9, 'Belarus'), (36, 5, 'Cambodia'), (37, 11, 'Cameroon'), (38, 1, 'Canada'), (41, 11, 'Central African Republic'), (42, 11, 'Chad'), (43, 3, 'Chile'), (44, 4, 'China'), (45, 3, 'Colombia'), (46, 11, 'Comoros'), (47, 11, 'Republic of the Congo'), (49, 2, 'Costa Rica'), (50, 9, 'Croatia'), (51, 2, 'Cuba'), (53, 8, 'Cyprus'), (54, 9, 'Czech Republic'), (55, 8, 'Denmark'), (56, 11, 'Djibouti'), (57, 2, 'Dominica')

In [ ]:
# Country 테이블에 INSERT 
for result_iterator in result:
    cursor.execute('insert into Country values (%s,%s,%s)',result_iterator)
    cursor.commit()
    print(result_iterator)
cursor.close()

(4, 6, 'Afghanistan')
(5, 9, 'Albania')
(6, 10, 'Algeria')
(7, 8, 'Andorra')
(8, 11, 'Angola')
(10, 2, 'Antigua and Barbuda')
(11, 3, 'Argentina')
(12, 7, 'Armenia')
(14, 12, 'Australia')
(15, 8, 'Austria')
(16, 7, 'Azerbaijan')
(17, 2, 'Bahamas')
(18, 10, 'Bahrain')
(19, 6, 'Bangladesh')
(20, 2, 'Barbados')
(21, 8, 'Belgium')
(22, 2, 'Belize')
(23, 11, 'Benin')
(25, 6, 'Bhutan')
(26, 3, 'Bolivia')
(28, 9, 'Bosnia-Herzegovina')
(29, 11, 'Botswana')
(30, 3, 'Brazil')
(31, 5, 'Brunei')
(32, 9, 'Bulgaria')
(33, 11, 'Burkina Faso')
(34, 11, 'Burundi')
(35, 9, 'Belarus')
(36, 5, 'Cambodia')
(37, 11, 'Cameroon')
(38, 1, 'Canada')
(41, 11, 'Central African Republic')
(42, 11, 'Chad')
(43, 3, 'Chile')
(44, 4, 'China')
(45, 3, 'Colombia')
(46, 11, 'Comoros')
(47, 11, 'Republic of the Congo')
(49, 2, 'Costa Rica')
(50, 9, 'Croatia')
(51, 2, 'Cuba')
(53, 8, 'Cyprus')
(54, 9, 'Czech Republic')
(55, 8, 'Denmark')
(56, 11, 'Djibouti')
(57, 2, 'Dominica')
(58, 2, 'Dominican Republic')
(59, 3, 'Ecuado

문제 3.  origin_terror_data 에서 attack type, target type, weapon type 관련 데이터는 code 와 txt 속성으로 정의되어 있습니다.  

문제 3-1. AttackType, TargetType, WeaponType 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고>
- AttackType.attacktype_code    = origin_terror_data.attacktype1
- AttackType.attacktype_desc    = origin_terror_data.attacktype1_txt 
- TargetType.targtype_code      = origin_terror_data.targtype1
- TargetType.targtype_desc      = origin_terror_data.targtype1_txt
- WeaponType.weaptype_code      = origin_terror_data.weaptype1
- WeaponType.weaptype_desc      = origin_terror_data.weaptype1_txt 

In [ ]:
# attacktype1 의 max length 체크 
cursor.execute('select max(length(attacktype1)) from origin_terror_data')

[(35,)]


In [ ]:
# AttackType 테이블 만들기 
cursor.execute('create table AttackType (attacktype_code int, attacktype_desc varchar(35), primary key (attacktype_code)')
cursor.execute('desc AttackType')
result = cursor.fetchall()
result

[('attacktype_code', b'int', 'NO', 'PRI', None, ''), ('attacktype_desc', b'varchar(35)', 'NO', '', None, '')]


In [ ]:
# targettype1 의 max length 체크 
cursor.execute('select max(length(targettype1)) from origin_terror_data')

[(30,)]


In [7]:
# TargetType 테이블 만들기 
cursor.execute('create table TargetType (targtype_code int, targtype_desc varchar(30), primary key (targtype_code)')
cursor.execute('desc TargetType')
result = cursor.fetchall()
result

In [ ]:
# weaptype1 의 max length 체크 
cursor.execute('select max(length(weaptype1)) from origin_terror_data')

[(75,)]


In [ ]:
# WeaponType 테이블 만들기 
cursor.execute('create table WeaponType (weaptype_code int, weaptype_desc varchar(75), primary key (weaptype_code)')
cursor.execute('desc WeaponType')
result = cursor.fetchall()
result

[('weaptype_code', b'int', 'NO', 'PRI', None, ''), ('weaptype_desc', b'varchar(80)', 'NO', '', None, '')]


문제 3-2. origin_terror_data 테이블에서 Attack Type, Target Type, Weapon Type 데이터를 추출하여 문제 3-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요.

제출 5. Table 생성 및 조회 결과 : DESC AttackType, DESC TargetType, WeaponType

제출 6. 관련 코드 및 실행 결과 (ipynb) 

In [ ]:
# attacktype1 데이터 추출하기 
cursor.execute('select distinct attacktype1, attacktype1_txt  from origin_terror_data')
result = cursor.fetchall()
result

[(1, 'Assassination'), (2, 'Armed Assault'), (3, 'Bombing/Explosion'), (4, 'Hijacking'), (5, 'Hostage Taking (Barricade Incident)'), (6, 'Hostage Taking (Kidnapping)'), (7, 'Facility/Infrastructure Attack'), (8, 'Unarmed Assault'), (9, 'Unknown')]


In [ ]:
# AttackType 테이블에 INSERT 
for result_iterator in result:
    cursor.execute('insert into AttackType  values (%s,%s)',result_iterator)
    cursor.commit()
    print(result_iterator)
cursor.close()

(1, 'Assassination')
(2, 'Armed Assault')
(3, 'Bombing/Explosion')
(4, 'Hijacking')
(5, 'Hostage Taking (Barricade Incident)')
(6, 'Hostage Taking (Kidnapping)')
(7, 'Facility/Infrastructure Attack')
(8, 'Unarmed Assault')
(9, 'Unknown')


In [ ]:
# targtype1 데이터 추출하기 
cursor.execute('select distinct targtype1, targtype1_txt  from origin_terror_data')
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)

(1, 'Business')
(2, 'Government (General)')
(3, 'Police')
(4, 'Military')
(5, 'Abortion Related')
(6, 'Airports & Aircraft')
(7, 'Government (Diplomatic)')
(8, 'Educational Institution')
(9, 'Food or Water Supply')
(10, 'Journalists & Media')
(11, 'Maritime')
(12, 'NGO')
(13, 'Other')
(14, 'Private Citizens & Property')
(15, 'Religious Figures/Institutions')
(16, 'Telecommunication')
(17, 'Terrorists/Non-State Militia')
(18, 'Tourists')
(19, 'Transportation')
(20, 'Unknown')
(21, 'Utilities')
(22, 'Violent Political Party')


In [ ]:
# TargetType 테이블에 INSERT 
for result_iterator in result:
    cursor.execute('insert into TargetType  values (%s,%s)',result_iterator)
    cursor.commit()
    print(result_iterator)
cursor.close()

# ALTER TABLE tablename
# CHANGE COLUMN old_columnname new_columnname new_datatype;

(1, 'Business')
(2, 'Government (General)')
(3, 'Police')
(4, 'Military')
(5, 'Abortion Related')
(6, 'Airports & Aircraft')
(7, 'Government (Diplomatic)')
(8, 'Educational Institution')
(9, 'Food or Water Supply')
(10, 'Journalists & Media')
(11, 'Maritime')
(12, 'NGO')
(13, 'Other')
(14, 'Private Citizens & Property')
(15, 'Religious Figures/Institutions')
(16, 'Telecommunication')
(17, 'Terrorists/Non-State Militia')
(18, 'Tourists')
(19, 'Transportation')
(20, 'Unknown')
(21, 'Utilities')
(22, 'Violent Political Party')


In [ ]:
# weaptype1 데이터 추출하기 
cursor.execute('select distinct weaptype1, weaptype1_txt  from origin_terror_data')
result = cursor.fetchall()
result

(1, 'Biological')
(2, 'Chemical')
(3, 'Radiological')
(5, 'Firearms')
(6, 'Explosives')
(7, 'Fake Weapons')
(8, 'Incendiary')
(9, 'Melee')
(10, 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)')
(11, 'Sabotage Equipment')
(12, 'Other')
(13, 'Unknown')


In [ ]:
# WeaponType 테이블에 INSERT 
for result_iterator in result:
    cursor.execute('insert into WeaponType  values (%s,%s)',result_iterator)
    cursor.commit()
    print(result_iterator)
cursor.close()

(1, 'Biological')
(2, 'Chemical')
(3, 'Radiological')
(5, 'Firearms')
(6, 'Explosives')
(7, 'Fake Weapons')
(8, 'Incendiary')
(9, 'Melee')
(10, 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)')
(11, 'Sabotage Equipment')
(12, 'Other')
(13, 'Unknown')


문제 4. TerrorData 테이블을 만들고 앞서 만들어둔 테이블과 관계를 설정하도록 하겠습니다. 
문제 4-1. TerrorData 테이블을 앞의 그림과 같이 생성하세요. 

- origin_terror_data 를 분석하여 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고> 
- TerrorData.terror id : Auto Increment 
- TerrorData.city_name    = origin_terror_data.city
- TerrorData.target       = orgin_terror_data.target1
- TerrorData.group_name   = origin_terror_data.gname
- TerrorData.kill_count   = origin_terror_data.nkill
- TerrorData.wound_count  = origin_terror_data.nwound
- TerrorData.motive       = origin_terror_data.motive
- TerrorData.summary      = origin_terror_data.summary
- TerrorData.latitude     = origin_terror_data_latitude
- TerrorData.longitude    = origin_terror_data_longitude
- TerrorData.terror_date  = origin_terror_data.iyear + origin_terror_Data.imonth + origin_terror_data.iday (Date Type)

**fetchall() , fetchone()**

In [ ]:
# max length 체크 

# city 의 max length 체크 
cursor.execute('select max(length(city)) from origin_terror_data')
city = cursor.fetchone()[0]

# target1 의 max length 체크 
cursor.execute('select max(length(target1)) from origin_terror_data')
target1 = cursor.fetchone()[0]

# gname 의 max length 체크 
cursor.execute('select max(length(gname)) from origin_terror_data')
gname = cursor.fetchone()[0]

# summary 의 max length 체크 
cursor.execute('select max(length(summary)) from origin_terror_data')
summary = cursor.fetchone()[0]

# motive 의 max length 체크 
cursor.execute('select max(length(motive)) from origin_terror_data')
motive = cursor.fetchone()[0]

print('city: {}'.format(city))
print('target1: {}'.format(target1))
print('gname: {}'.format(gname))
print('summary: {}'.format(summary))
print('motive: {}'.format(motive))


city : 65
target1 : 343
gname : 112
summary : 2431
motive : 899
city : 65
target1 : 343
gname : 112
summary : 2431
motive : 899


In [ ]:
# TerrorData 테이블 만들기 
cursor.execute("""create table TerrorData 
               (terror_id int auto increment, 
               terror_date date,
               region_code int,
               country_code int,
               city varchar(65),
               target varchar(343),
               group_name varchar(112),
               targtype_code int,
               attacktype_code int,
               weaptype_code inbt,
               kill_count int,
               wound_count int ,
               motive varchar(900),
               summary (2450),
               latitude decimal(16,14),
               longitude decimal(17,14),
               primary key (terror_id),
               foreign key (region_code) references Region(region_code),
               foreign key (country_code) references Country(country_code),
               foreign key (attacktype_code) references AttackType(attacktype_code),
               foreign key (targtype_code) references TargetType(targtype_code),
               foreign key (weaptype_code) references WeaponType(weaptype_code)""")

cursor.execute('desc TerrorData')
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)

('terror_id', b'int', 'NO', 'PRI', None, 'auto_increment')
('terror_date', b'date', 'NO', '', None, '')
('region_code', b'int', 'NO', 'MUL', None, '')
('country_code', b'int', 'NO', 'MUL', None, '')
('city', b'varchar(65)', 'YES', '', None, '')
('target', b'varchar(350)', 'YES', '', None, '')
('group_name', b'varchar(120)', 'YES', '', None, '')
('targtype_code', b'int', 'NO', 'MUL', None, '')
('attacktype_code', b'int', 'NO', 'MUL', None, '')
('weaptype_code', b'int', 'NO', 'MUL', None, '')
('kill_count', b'int', 'NO', '', b'0', '')
('wound_count', b'int', 'NO', '', b'0', '')
('motive', b'varchar(900)', 'YES', '', None, '')
('summary', b'varchar(2450)', 'YES', '', None, '')
('latitude', b'decimal(16,14)', 'YES', '', None, '')
('longitude', b'decimal(17,14)', 'YES', '', None, '')


문제 4-2. origin_terror_data 테이블에서 Terror Data를 추출하여 문제 4-1.에서 생성한 테이블에 입력하고 확인하세요.

- 앞서 생성한 Region, Country, AttackType, TargetType, WeaponType 데이터와의 관계에 주의하세요.
- Count 값을 가지는 칼럼의 값이 null 인 경우, 0으로 예외처리 해주세요.
- 위도 경도 데이터 중 범위를 넘어서는 데이터가 존재합니다. 이 경우, null 값으로 예외처리 해주세요. (위도 경도 범위 : 구글링해보세요.)
- terror_date 칼럼의 경우, origin_terror_data 의 연, 월, 일 정보를 조합하여 date type 으로 정의해주세요. (Format : ‘YYYY-mm-dd’)
- origin_terror_data 의 월, 일 정보중 값이 0 인 경우 date type 으로 변환되지 않습니다. 이 경우, 1 로 예외처리 해주세요.
- 데이터 입력까지 완료한 이후, origin_terror_data 테이블을 삭제하고 확인하세요.

제출 7. Table 생성 및 조회 결과 : DESC TerrorData, SELECT COUNT(*) FROM TerrorData 

제출 8. 관련 코드 및 실행 결과 (ipynb)


- IF()문, 
- CASE WHEN~ END, 
- STR_TO_DATE(CONCAT(),%Y-%m-%d), 
- CAST( AS 데이터 타입)

In [ ]:
# data 전처리 
sql = ("""INSERT INTO TerrorData (region_code, country_code, attacktype_code, 
       targtype_code, weaptype_code, city_name, target, group_name, kill_count,
        wound_count, motive, summary, latitude, longitude, terror_date)
       SELECT region, country, attacktype1,
       targtype1, weaptype1, city, target1, gname, 
       If (nkill is null, 0 ,nkill),
       IF (nwound is null, 0 ,nwound),
       motive, summary,
       CASE  WHEN latitude BETWEEN -90 AND 90 THEN latitude ELSE NUL END,
       CASE WHEN longitude BETWEEN -180 AND 180 THEN longitude ELSE NULL END,
       STR_TO_DATE(CONCAT(IF(iyear = 0, 1, iyear), '-',IF(imonth = 0, 1, imonth), '-',
       IF(iday = 0, 1, iday)), '%Y-%m-%d'))""")
       
# CAST(IF(iyear =0 ,1,iyear)%IF(imonth =0 ,1, imonth)%IF(iday =0,1,iday) AS DATE))

cursor.execute(sql)


In [ ]:
# TerrorData 테이블에 INSERT 



[(181691,)]


문제 5. TerrorData 의 전체 기간에서 테러의 숫자를 연도별로 집계하여 연도별 테러 발생 건수를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 

제출 9. 관련 코드 및 실행 결과 (ipynb)

**EXTRACT( year from DATE)**
- 날짜에서 년도만 추출

In [ ]:
sql = ("""select EXTRACT(year from terror_data), count(*) from TerrorData group by terror_data
       """)

cursor.execute(sql)
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)


('1970', 651)
('1971', 471)
('1972', 568)
('1973', 473)
('1974', 581)
('1975', 740)
('1976', 923)
('1977', 1319)
('1978', 1526)
('1979', 2662)
('1980', 2662)
('1981', 2586)
('1986', 2860)
('1982', 2544)
('1983', 2870)
('1984', 3495)
('1985', 2915)
('1987', 3183)
('1988', 3721)
('1989', 4324)
('1990', 3887)
('1991', 4683)
('1992', 5071)
('1994', 3456)
('1995', 3081)
('1996', 3058)
('1997', 3197)
('1998', 934)
('1999', 1395)
('2000', 1814)
('2001', 1906)
('2002', 1333)
('2003', 1278)
('2004', 1166)
('2005', 2017)
('2006', 2758)
('2007', 3242)
('2008', 4805)
('2009', 4721)
('2010', 4826)
('2011', 5076)
('2012', 8522)
('2013', 12036)
('2014', 16903)
('2015', 14965)
('2016', 13587)
('2017', 10900)


문제 6. TerrorData 에서 테러가 가장 많이 일어난 순서로 국가를 정렬하여 상위 10위 국가를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 10. 관련 코드 및 실행 결과 (ipynb)

In [ ]:
# 테러가 많이 일어난 상위 10위 Region
cursor.execute("""select R.region_name, T.count(*) from TerrorData as T 
               LEFT JOIN Region as R ON R.region_code = T.region_code
               group by R.region_name
               order by T.count(*) desc limit 10""")
region = cursor.fetchall()
for result_iterator in region:
    print(result_iterator)



('Middle East & North Africa', 50474)
('South Asia', 44974)
('South America', 18978)
('Sub-Saharan Africa', 17550)
('Western Europe', 16639)
('Southeast Asia', 12485)
('Central America & Caribbean', 10344)
('Eastern Europe', 5144)
('North America', 3456)
('East Asia', 802)
('Iraq', 24636)
('Pakistan', 14368)
('Afghanistan', 12731)
('India', 11960)
('Colombia', 8306)
('Philippines', 6908)
('Peru', 6096)
('El Salvador', 5320)
('United Kingdom', 5235)
('Turkey', 4292)


In [ ]:
# 테러가 많이 일어난 상위 10위 Country 
cursor.execute("""select C.country_name, T.count(*) from TerrorData as T 
               LEFT JOIN Country as C ON C.country_code = T.country_code
               group by C.country_name
               order by T.count(*) desc limit 10""")
country = cursor.fetchall()
for result_iterator in country:
    print(result_iterator)

문제 7. TerrorData 에서 테러가 가장 많이 일어난 상위 10위 국가에 대해 국가별로 사망자수와 부상자수, 사상자수(사망자수 + 부상자수)를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 11. 관련 코드 및 실행 결과 (ipynb)

In [ ]:
cursor.execute("""select C.country_name, T.count(*),sum(T.kill_count),sum(T.wound_count),
               sum(T.kill_count+T.wound_count)
               from TerrorData as T 
               LEFT JOIN Country as C ON C.country_code = T.country_code
               group by C.country_name
               order by T.count(*) desc limit 10 
               """)
country = cursor.fetchall()
for result_iterator in country:
    print(result_iterator)

('Iraq', 24636, Decimal('78589'), Decimal('134690'), Decimal('213279'))
('Pakistan', 14368, Decimal('23822'), Decimal('42038'), Decimal('65860'))
('Afghanistan', 12731, Decimal('39384'), Decimal('44277'), Decimal('83661'))
('India', 11960, Decimal('19341'), Decimal('28980'), Decimal('48321'))
('Colombia', 8306, Decimal('14698'), Decimal('10328'), Decimal('25026'))
('Philippines', 6908, Decimal('9559'), Decimal('13367'), Decimal('22926'))
('Peru', 6096, Decimal('12771'), Decimal('4078'), Decimal('16849'))
('El Salvador', 5320, Decimal('12053'), Decimal('5062'), Decimal('17115'))
('United Kingdom', 5235, Decimal('3410'), Decimal('6106'), Decimal('9516'))
('Turkey', 4292, Decimal('6888'), Decimal('9899'), Decimal('16787'))


문제 8. 지역별 테러 공격 형태에 따른 사망자수, 부상자수, 사상자수를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 지역 정보(Region)는 이름으로 출력하세요.

제출 12. 관련 코드 및 실행 결과 (ipynb)

**테이블 조인은 하나의 테이블에 다수의 테이블도 가능**
- 필요한 만큼 JOIN 절을 추가하면 여러 개의 테이블을 조인할 수 있습니다

In [ ]:
# 지역별 테러 공격 형태에 따른 사망자와 사상자 수 
cursor.execute("""select R.region_name,T.count(*), A.attack_desc,
               sum(T.kill_count),sum(T.wound_count),
               sum(T.kill_count+T.wound_count)
               from TerrorData as T 
               LEFT JOIN Country as C ON C.country_code = T.country_code
               LEFT JOIN AttackType as A ON A.attacktype_code = T.attacktype_code               
               group by R.region_name, A.attack_desc
               order by T.count(*) desc 
               """)
region = cursor.fetchall()
for result_iterator in region:
    print(result_iterator)


('Central America & Caribbean', 'Assassination', 1254, 1254)
('Southeast Asia', 'Assassination', 1369, 1369)
('South America', 'Assassination', 2745, 2745)
('North America', 'Assassination', 255, 255)
('Middle East & North Africa', 'Assassination', 4206, 4206)
('South Asia', 'Assassination', 4301, 4301)
('Western Europe', 'Assassination', 2938, 2938)
('Sub-Saharan Africa', 'Assassination', 1638, 1638)
('Australasia & Oceania', 'Assassination', 36, 36)
('East Asia', 'Assassination', 55, 55)
('Eastern Europe', 'Assassination', 400, 400)
('Central Asia', 'Assassination', 115, 115)
('North America', 'Armed Assault', 448, 448)
('South America', 'Armed Assault', 3875, 3875)
('Western Europe', 'Armed Assault', 1724, 1724)
('Middle East & North Africa', 'Armed Assault', 9273, 9273)
('Central America & Caribbean', 'Armed Assault', 4361, 4361)
('Southeast Asia', 'Armed Assault', 4022, 4022)
('Australasia & Oceania', 'Armed Assault', 51, 51)
('Sub-Saharan Africa', 'Armed Assault', 6004, 6004)
('E

---

# 참고사항
- 1번문제는 아래 오류가 해결되지 않아 2번부터 진행하였습니다.
PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b) 

# reference

민형동 (2020). Global Terrorism Data를 활용한 국가별 테러유형 분석. 한국공안행정학회, 82, 63-84. 

한인택 (2017). Global Terrorism Database를 활용한 테러리즘의 현황분석. 

민형기 (2019). 전세계 테러 데이터 분석 (따라하기). https://pinkwink.kr/1214 

ASHWINI SWSAIN (2018). Terrorism Around The World. https://www.kaggle.com/code/ash316/terrorism-around-the-world/notebook

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021). *Global Terrorism Database* (GTD) [Data set]. University of Maryland. https://www.start.umd.edu/gtd

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021, August). *Global Terrorism Database codebook: Methodology, inclusion criteria, and variables.* University of Maryland. https://www.start.umd.edu/gtd/downloads/Codebook.pdf 
